In [ ]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import json
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from zipfile import ZipFile
from sklearn.model_selection import train_test_split


In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
path='/content/plantvillage-dataset/plantvillage dataset/color'
data_classes=os.listdir(path)
print(data_classes)

In [ ]:
class_nam=[]
class_img=[]
image_paths=[]
min_sample=7
for item in data_classes:
    class_path=os.path.join(path,item)
    class_list=os.listdir(class_path)

    if len(class_list)>=min_sample:
        class_nam.append(item)
        for image in class_list:
            class_img.append((item,image,os.path.join(class_path,image)))
            image_path=os.path.join(class_path,image)
            image_paths.append(image_path)
    else:
        print(item, ' has only', len(class_list),
              ' so it was removed')
print(len(class_nam))
print(image_paths[:5])

In [ ]:
df=pd.DataFrame(data=class_img,columns=['Label','Images','Path'])
df.info()

In [ ]:
print('Total number of images:',len(df))
label_count=df['Label'].value_counts()
print(label_count)

In [ ]:
df.head()

In [ ]:
class_counts = df['Label'].value_counts()

plt.figure(figsize=(12, 8))
ax = sns.barplot(x=class_counts.values, y=class_counts.index, orient='h')
plt.title('Class Distribution')
plt.xlabel('Number of Images')
plt.ylabel('Plant Types')
plt.tight_layout() 


for i, v in enumerate(class_counts.values):
    ax.text(v + 5, i, str(v), color='black', va='center')

plt.show()

In [ ]:
img_path = df.Path[0]
img = mpimg.imread(img_path)
plt.imshow(img)


In [ ]:
img_size = 224
batch_size = 32

In [ ]:
data_gen = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2
)

In [ ]:
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

In [ ]:
train_data = data_gen.flow_from_directory(
    path,
    target_size = (img_size,img_size),
    batch_size = batch_size,
    subset= 'training',
    class_mode = 'categorical'
)

In [ ]:
test_data = data_gen.flow_from_directory(
    path,
    target_size = (img_size,img_size),
    batch_size = batch_size,
    subset= 'validation',
    class_mode = 'categorical'
)

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3,3),activation='relu', input_shape=(img_size, img_size, 3)))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Conv2D(64, (3,3),activation='relu'))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Flatten())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(train_data.num_classes,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

In [ ]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_data,
    steps_per_epoch = train_data.samples // batch_size,
    epochs = 15,
    validation_data = test_data,
    validation_steps = test_data.samples // batch_size,
    callbacks=[early_stop, reduce_lr]
)

In [ ]:
print("Evalutind Model")
val_loss, val_accuracy = model.evaluate(test_data, steps = test_data.samples // batch_size)
print(f"Validation score is {val_accuracy * 100:.2f}%")

In [ ]:
def load_and_preprocess_image(img_path, target_size=(224,224)):
    img = Image.open(img_path)
    img = img.resize(target_size)

    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array.astype('float32') / 255.
    return img_array


In [ ]:
def predict_image(model, img_path, class_indices):
    preprocessed_img = load_and_preprocess_image(img_path)
    prediction = model.predict(preprocessed_img)
    predicted_class_index = np.argmax(prediction, axis=1)[0]
    predicted_class_name = class_indices[predicted_class_index]
    return predicted_class_name


In [ ]:
class_indices = {v : k for k, v in train_data.class_indices.items()}

In [ ]:
class_indices

In [ ]:
json.dump(class_indices, open('class_indices.json', 'w'))

In [ ]:
img_path = '/content/merged_dataset/Flag Smut/0281.jpg'
#img_path = '/content/plant-disease-expert/Image Data base/Image Data base/Blueberry healthy/Blueberry_healthy10.jpg'
#img_path = '/content/plant-disease-expert/Image Data base/Image Data base/Brown spot in rice leaf/Brown_spot_in_rice_leaf1.jpg'



In [ ]:
result = predict_image(model, img_path, class_indices)

print("predicted disease of palnt is", result)

In [ ]:
import pickle

In [ ]:
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)